## ChatNS

In [1]:
def parse_map(map_str):
    return [list(map(int, line.strip())) for line in map_str.splitlines()]

def find_trailheads(map_data):
    trailheads = []
    for r, row in enumerate(map_data):
        for c, height in enumerate(row):
            if height == 0:
                trailheads.append((r, c))
    return trailheads

def is_valid_move(map_data, r, c, current_height):
    return 0 <= r < len(map_data) and 0 <= c < len(map_data[0]) and map_data[r][c] == current_height + 1

def bfs(map_data, trailhead):
    from collections import deque
    queue = deque([(trailhead[0], trailhead[1], 0)])  # (row, col, current height)
    reached_nines = set()

    while queue:
        r, c, cur_height = queue.popleft()

        # If we reach a height of 9, count this position
        if cur_height == 9:
            reached_nines.add((r, c))
            continue

        # Attempt to move in each of the four directions
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nr, nc = r + dr, c + dc
            if is_valid_move(map_data, nr, nc, cur_height):
                queue.append((nr, nc, cur_height + 1))

    return len(reached_nines)

def calculate_trailhead_scores(map_data):
    scores = 0
    trailheads = find_trailheads(map_data)

    for trailhead in trailheads:
        score = bfs(map_data, trailhead)
        scores += score

    return scores

# Example topographic map input
map_str = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

# Parse map and calculate scores
map_data = parse_map(map_str)
total_score = calculate_trailhead_scores(map_data)
print(f"The sum of the scores of all trailheads is: {total_score}")

The sum of the scores of all trailheads is: 36


In [5]:
with open("./data/Day 10/input.txt") as f:
    map_str = f.read()

In [3]:
map_data = parse_map(map_str)
total_score = calculate_trailhead_scores(map_data)
print(f"The sum of the scores of all trailheads is: {total_score}")

The sum of the scores of all trailheads is: 822


In [4]:
def parse_map(map_str):
    return [list(map(int, line.strip())) for line in map_str.splitlines()]


def find_trailheads(map_data):
    trailheads = []
    for r, row in enumerate(map_data):
        for c, height in enumerate(row):
            if height == 0:
                trailheads.append((r, c))
    return trailheads


def calculate_trailhead_ratings(map_data):
    rows, cols = len(map_data), len(map_data[0])
    ratings = 0
    trailheads = find_trailheads(map_data)

    def dfs(r, c, current_height, memo):
        # If current position is a 9, return 1 as we found a path to a 9
        if current_height == 9:
            return 1
        if (r, c, current_height) in memo:
            return memo[(r, c, current_height)]

        trail_count = 0
        # Attempt to move in each of the four directions
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nr, nc = r + dr, c + dc
            if (
                0 <= nr < rows
                and 0 <= nc < cols
                and map_data[nr][nc] == current_height + 1
            ):
                trail_count += dfs(nr, nc, current_height + 1, memo)

        memo[(r, c, current_height)] = trail_count
        return trail_count

    for trailhead in trailheads:
        r, c = trailhead
        memo = {}
        trailhead_rating = dfs(r, c, 0, memo)
        ratings += trailhead_rating

    return ratings


# Example topographic map input
map_str = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

# Parse map and calculate ratings
map_data = parse_map(map_str)
total_ratings = calculate_trailhead_ratings(map_data)
print(f"The sum of the ratings of all trailheads is: {total_ratings}")

The sum of the ratings of all trailheads is: 81


In [7]:
map_data = parse_map(map_str)
total_ratings = calculate_trailhead_ratings(map_data)
print(f"The sum of the ratings of all trailheads is: {total_ratings}")

The sum of the ratings of all trailheads is: 1801


## Part 1

In [3]:
example_map_str = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

In [4]:
import numpy as np
def parse_input_map(map_str: str) -> np.array:
    return np.array([list(map(int, line.strip())) for line in map_str.splitlines()])

In [5]:
example_map = parse_input_map(example_map_str)
with open("./data/Day 10/input.txt") as f:
    input_map_str = f.read()
real_map = parse_input_map(input_map_str)

In [51]:
import networkx as nx

effect_map = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

def traversable_paths(map, start_node, goal=9):
    paths = [(int(map[start_node]), start_node)]
    for direction in ["up", "right", "down", "left"]:
        new_node = (start_node[0] + effect_map[direction][0], start_node[1] + effect_map[direction][1])
        if 0 <= new_node[0] < map.shape[0] and 0 <= new_node[1] < map.shape[1] and map[new_node] == map[start_node] + 1:
            if map[new_node] == goal:
                paths.append((int(map[new_node]), new_node))
            else:
                paths += traversable_paths(map, new_node, goal)


    return paths

def build_map_graph(map, start_node, goal=9):
    G = nx.DiGraph()
    G.add_node(start_node, height=int(map[start_node]))
    for direction in ["up", "right", "down", "left"]:
        new_node = (start_node[0] + effect_map[direction][0], start_node[1] + effect_map[direction][1])
        if 0 <= new_node[0] < map.shape[0] and 0 <= new_node[1] < map.shape[1] and map[new_node] == map[start_node] + 1:
            G.add_node(new_node, height=int(map[new_node]))
            G.add_edge(start_node, new_node)
            if map[new_node] != goal:
                G = nx.compose(G, build_map_graph(map, new_node, goal))
    return G

def get_rating(map, start_node, goal=9):
    G = build_map_graph(map, start_node, goal)
    targets = [k for k,v in nx.get_node_attributes(G, "height").items() if v == goal]
    rating = 0
    for target in targets:
        rating += len(list(nx.all_simple_paths(G, start_node, target)))
    return rating

def all_paths_from_0_to_9(map):
    start_locs = np.argwhere(map == 0)
    rating = 0
    for loc in start_locs:
        loc = tuple([int(x) for x in loc])
        start_rating = get_rating(map, tuple(loc))
        rating += start_rating
        print(f"Rating from {loc} to 9: {start_rating}")
    return rating

In [46]:
start_locs = np.argwhere(example_map == 0)
total_score = 0
for loc in start_locs:
    paths = traversable_paths(example_map, tuple(loc))
    paths_to_9 = len(set(j for i, j in paths if i == 9))
    total_score += paths_to_9
print(f"Total score: {total_score}")

Total score: 36


In [47]:
start_locs = np.argwhere(real_map == 0)
total_score = 0
for loc in start_locs:
    paths = traversable_paths(real_map, tuple(loc))
    paths_to_9 = len(set(j for i, j in paths if i == 9))
    total_score += paths_to_9
print(f"Total score: {total_score}")

Total score: 822


## Part 2

In [52]:
all_paths_from_0_to_9(example_map)

Rating from (0, 2) to 9: 20
Rating from (0, 4) to 9: 24
Rating from (2, 4) to 9: 10
Rating from (4, 6) to 9: 4
Rating from (5, 2) to 9: 1
Rating from (5, 5) to 9: 4
Rating from (6, 0) to 9: 5
Rating from (6, 6) to 9: 8
Rating from (7, 1) to 9: 5


81

In [53]:
all_paths_from_0_to_9(real_map)

Rating from (0, 3) to 9: 7
Rating from (0, 22) to 9: 3
Rating from (0, 33) to 9: 1
Rating from (0, 37) to 9: 6
Rating from (0, 39) to 9: 6
Rating from (0, 42) to 9: 1
Rating from (0, 43) to 9: 6
Rating from (0, 47) to 9: 9
Rating from (1, 4) to 9: 7
Rating from (1, 7) to 9: 1
Rating from (2, 2) to 9: 1
Rating from (2, 16) to 9: 11
Rating from (2, 18) to 9: 12
Rating from (2, 57) to 9: 6
Rating from (3, 31) to 9: 6
Rating from (3, 44) to 9: 4
Rating from (3, 49) to 9: 7
Rating from (4, 13) to 9: 4
Rating from (4, 14) to 9: 5
Rating from (4, 20) to 9: 5
Rating from (4, 22) to 9: 1
Rating from (4, 46) to 9: 3
Rating from (4, 56) to 9: 2
Rating from (5, 8) to 9: 2
Rating from (5, 25) to 9: 2
Rating from (5, 29) to 9: 3
Rating from (5, 31) to 9: 7
Rating from (5, 44) to 9: 18
Rating from (5, 51) to 9: 9
Rating from (6, 20) to 9: 9
Rating from (6, 24) to 9: 2
Rating from (6, 39) to 9: 5
Rating from (6, 41) to 9: 1
Rating from (6, 50) to 9: 9
Rating from (6, 57) to 9: 1
Rating from (7, 2) to 

1801